In [1]:
%matplotlib inline

In [2]:
import os
import asyncio
import numpy as np
import matplotlib.pyplot as plt
import mdtraj as mdt
import MDAnalysis as mda
import arcd
import arcd.distributed as arcdd

Tensorflow/Keras not available


In [3]:
# setup logging
# executing this file sets the variable LOGCONFIG, which is a dictionary of logging presets 
%run ../resources/logconf.py

In [4]:
LOGCONFIG

{'version': 1,
 'disable_existing_loggers': False,
 'formatters': {'standardFormatter': {'class': 'logging.Formatter',
   'format': '(%(levelname)s)%(name)s: %(message)s'},
  'msgOnly': {'class': 'logging.Formatter', 'format': '%(message)s'}},
 'handlers': {'stdout': {'class': 'logging.StreamHandler',
   'level': 'NOTSET',
   'formatter': 'msgOnly'},
  'warnout': {'class': 'logging.StreamHandler',
   'level': 'WARN',
   'formatter': 'standardFormatter'},
  'initf': {'class': 'logging.FileHandler',
   'level': 'INFO',
   'mode': 'w',
   'filename': 'initialization.log',
   'formatter': 'standardFormatter'},
  'stdf': {'class': 'logging.FileHandler',
   'level': 'INFO',
   'mode': 'w',
   'filename': 'simulation.log',
   'formatter': 'standardFormatter'}},
 'loggers': {'': {'level': 'INFO', 'handlers': ['stdf', 'warnout']},
  'openpathsampling.initialization': {'level': 'INFO',
   'handlers': ['initf'],
   'qualname': 'openpathsampling.initialization',
   'propagate': 0}}}

In [5]:
# you can either modify single values or use it as is to get the same setupt as in the OPS default logging config file
# you could e.g. do LOGCONF['handlers']['stdf']['filename'] = new_name to change the filename of the log
# the default is to create 'simulation.log' and 'initialization.log' in the current working directory
import logging.config
LOGCONFIG["handlers"]["stdf"]["filename"] = "simulation_committors.log"
LOGCONFIG["handlers"]["initf"]["filename"] = "initlog_committors.log"
logging.config.dictConfig(LOGCONFIG)

# now the actual setup

In [6]:
scratch_dir = "/home/think/scratch/arcd_distributed/"

In [7]:
# states definition
def alpha_R(traj):
    traj = mdt.load(traj.trajectory_file, 
                    top=os.path.join(scratch_dir, "gmx_infiles/conf.gro"),  # mdt can not work with tprs, so we use theinitial gro for now
                    )
    psi = mdt.compute_dihedrals(traj, indices=[[6,8,14,16]])[:, 0]
    phi = mdt.compute_dihedrals(traj, indices=[[4,6,8,14]])[:, 0]
    state = np.full_like(psi, False, dtype=bool)
    # phi: -pi -> 0 
    # psi: > -50 but smaller 30 degree
    deg = 180/np.pi
    state[(phi <= 0) & (-50/deg <= psi) & (psi <= 30/deg)] = True
    return state


wrapped_alphaR = arcdd.trajectory.TrajectoryFunctionWrapper(alpha_R)


def C7_eq(traj):
    traj = mdt.load(traj.trajectory_file, 
                    top=os.path.join(scratch_dir, "gmx_infiles/conf.gro"),  # mdt can not work with tprs, so we use theinitial gro for now
                    )
    psi = mdt.compute_dihedrals(traj, indices=[[6,8,14,16]])[:, 0]
    phi = mdt.compute_dihedrals(traj, indices=[[4,6,8,14]])[:, 0]
    state = np.full_like(psi, False, dtype=bool)
    # phi: -pi -> 0 
    # psi: 120 -> 200 degree
    deg = 180/np.pi
    state[(phi <= 0) & ((120/deg <= psi) | (-160/deg >= psi))] = True
    return state


wrapped_C7_eq = arcdd.TrajectoryFunctionWrapper(C7_eq)

In [8]:
init_confs = [(arcdd.Trajectory(os.path.join(scratch_dir, "gmx_infiles/tp_frame.gro"),
                                os.path.join(scratch_dir, "gmx_infiles/tp_frame.gro")
                                ), 0) for _ in range(4)]
# engine_kwargs
gro = os.path.join(scratch_dir, "gmx_infiles/conf.gro")
top = os.path.join(scratch_dir, "gmx_infiles/topol.top")
mdp = arcdd.MDP(os.path.join(scratch_dir, "gmx_infiles/md.mdp"))

engine_kwargs = {"gro_file":gro, "top_file":top, "mdrun_extra_args":"-nt 2"}

simulation = arcdd.CommittorSimulation(workdir=os.path.join(scratch_dir, "committors_TwoWay"),
                                       starting_configurations=init_confs,
                                       states=[wrapped_C7_eq, wrapped_alphaR],
                                       engine_cls=arcdd.GmxEngine,
                                       engine_kwargs=engine_kwargs,
                                       engine_mdconfig=mdp,
                                       T=mdp["ref-t"][0],
                                       walltime_per_part=0.01,
                                       n_max_concurrent=5,
                                       two_way=True,
                                       )

In [9]:
ret = await simulation.run(25)

/home/think/.conda/envs/arcd_dev_3_2021/lib/python3.8/site-packages/MDAnalysis/topology/base.py:203: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  residx = np.zeros_like(criteria[0], dtype=np.int)
/home/think/.conda/envs/arcd_dev_3_2021/lib/python3.8/site-packages/MDAnalysis/coordinates/base.py:865: UserWarning: Reader has no dt information, set to 1.0 ps
  warnings.warn("Reader has no dt information, set to 1.0 ps")
(WARNING)arcd.distributed.logic: MD engine crashed. Retrying committor step: Configuration 0, shot 1.
/home/think/.conda/envs/arcd_

EngineCrashedError: Non-zero exit code from mdrun.

/home/think/.conda/envs/arcd_dev_3_2021/lib/python3.8/site-packages/MDAnalysis/topology/base.py:203: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  residx = np.zeros_like(criteria[0], dtype=np.int)
/home/think/.conda/envs/arcd_dev_3_2021/lib/python3.8/site-packages/MDAnalysis/coordinates/base.py:865: UserWarning: Reader has no dt information, set to 1.0 ps
  warnings.warn("Reader has no dt information, set to 1.0 ps")
(WARNING)arcd.distributed.logic: MD engine crashed. Retrying committor step: Configuration 1, shot 10.
/home/think/.conda/envs/arcd

In [10]:
ret

NameError: name 'ret' is not defined

/home/think/.conda/envs/arcd_dev_3_2021/lib/python3.8/site-packages/MDAnalysis/topology/base.py:203: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  residx = np.zeros_like(criteria[0], dtype=np.int)
/home/think/.conda/envs/arcd_dev_3_2021/lib/python3.8/site-packages/MDAnalysis/coordinates/base.py:865: UserWarning: Reader has no dt information, set to 1.0 ps
  warnings.warn("Reader has no dt information, set to 1.0 ps")
/home/think/.conda/envs/arcd_dev_3_2021/lib/python3.8/site-packages/MDAnalysis/topology/base.py:203: DeprecationWarning: `np.int` i